In [1]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup
import json
import time
from tqdm.notebook import tqdm
import PyBinance
from datetime import datetime, timedelta

## 1.1 Setup

In [2]:
top_symbols = pd.read_csv("../Meta-Work/Inference/Top-Symbols.csv")
top_symbols['Base-Coin'] = top_symbols['symbol_id'].str.split("_").str[2]
top_symbols['Quote-Coin'] = top_symbols['symbol_id'].str.split("_").str[3]
top_symbols['Symbol'] = top_symbols['Base-Coin'] + top_symbols['Quote-Coin']
print(top_symbols.shape)
top_symbols.head()

(204, 4)


,symbol_id,Base-Coin,Quote-Coin,Symbol
0,BINANCE_SPOT_BTC_USDT,BTC,USDT,BTCUSDT
1,BINANCE_SPOT_ETH_USDT,ETH,USDT,ETHUSDT
2,BINANCE_SPOT_BNB_USDT,BNB,USDT,BNBUSDT
3,BINANCE_SPOT_ETH_BTC,ETH,BTC,ETHBTC
4,BINANCE_SPOT_BCH_USDT,BCH,USDT,BCHUSDT


In [3]:
response = requests.get("https://free-proxy-list.net/")
soup = BeautifulSoup(response.text, 'html.parser')
rows = soup.find_all('tr')
proxy_df = []
for i in range(1, 300+1): # all other indices of tr are irrelevant
    proxy = [td.text for td in rows[i].find_all('td')]
    proxy_df.append(proxy)
columns = [th.text for th in rows[0].find_all('th')]
proxy_df = pd.DataFrame(proxy_df, columns=columns)
print(proxy_df.shape)
proxy_df.head()

(300, 8)


,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked
0,3.249.236.176,3128,IE,Ireland,elite proxy,no,yes,1 minute ago
1,200.206.50.66,42515,BR,Brazil,elite proxy,no,yes,1 minute ago
2,102.164.202.75,39933,ZA,South Africa,elite proxy,no,yes,1 minute ago
3,190.152.19.62,36711,EC,Ecuador,elite proxy,no,yes,1 minute ago
4,104.148.76.179,3128,US,United States,elite proxy,no,yes,1 minute ago


In [4]:
base_api = 'https://api.binance.com'
exchange_info_api = base_api + "/api/v3/exchangeInfo"
symbols_df = json.loads(requests.get(exchange_info_api).text)['symbols']
symbols_df = pd.DataFrame(symbols_df)
print(symbols_df.shape)
symbols_df.head()

(800, 17)


,symbol,status,baseAsset,baseAssetPrecision,quoteAsset,quotePrecision,quoteAssetPrecision,baseCommissionPrecision,quoteCommissionPrecision,orderTypes,icebergAllowed,ocoAllowed,quoteOrderQtyMarketAllowed,isSpotTradingAllowed,isMarginTradingAllowed,filters,permissions
0,ETHBTC,TRADING,ETH,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","[SPOT, MARGIN]"
1,LTCBTC,TRADING,LTC,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","[SPOT, MARGIN]"
2,BNBBTC,TRADING,BNB,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","[SPOT, MARGIN]"
3,NEOBTC,TRADING,NEO,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","[SPOT, MARGIN]"
4,QTUMETH,TRADING,QTUM,8,ETH,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,True,False,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[SPOT]


In [5]:
print(top_symbols.shape)
mask = top_symbols['Symbol'].isin(symbols_df['symbol'])
print(top_symbols[np.invert(mask)])

(204, 4)
                  symbol_id Base-Coin Quote-Coin    Symbol
25   BINANCE_SPOT_ETHOS_BTC     ETHOS        BTC  ETHOSBTC
30   BINANCE_SPOT_METAL_BTC     METAL        BTC  METALBTC
79   BINANCE_SPOT_YOYOW_BTC     YOYOW        BTC  YOYOWBTC
81   BINANCE_SPOT_COMET_BTC     COMET        BTC  COMETBTC
98   BINANCE_SPOT_METAL_ETH     METAL        ETH  METALETH
123  BINANCE_SPOT_ETHOS_ETH     ETHOS        ETH  ETHOSETH
139  BINANCE_SPOT_COMET_ETH     COMET        ETH  COMETETH
185  BINANCE_SPOT_YOYOW_ETH     YOYOW        ETH  YOYOWETH
187  BINANCE_SPOT_COMET_BNB     COMET        BNB  COMETBNB
199  BINANCE_SPOT_YOYOW_BNB     YOYOW        BNB  YOYOWBNB


In [6]:
top_symbols = top_symbols[mask]
print(len(top_symbols))

194


In [7]:
pd.DataFrame(json.loads(requests.get(exchange_info_api).text)['rateLimits'])

,rateLimitType,interval,intervalNum,limit
0,REQUEST_WEIGHT,MINUTE,1,1200
1,ORDERS,SECOND,10,100
2,ORDERS,DAY,1,200000


## 1.2 Define Data Puller

In [8]:
%%writefile PyBinance.py

import numpy as np
import pandas as pd
import requests
import json
from tqdm.notebook import tqdm
from datetime import datetime, timedelta

class BinanceKlinesPuller:
    
    def __init__(self, base_coin, quote_coin, start_time, end_time, interval):
        self.base_api = 'https://api.binance.com'
        self.klines_api = self.base_api + "/api/v3/klines?symbol={}&interval={}&startTime={}&endTime={}&limit=1000"
        self.base_coin = base_coin
        self.quote_coin = quote_coin
        self.symbol = self.base_coin + self.quote_coin
        self.start_time = start_time
        self.end_time = end_time
        self.interval = interval
        self.interval_pandas = self.interval.upper()
        if self.interval_pandas[-1] == "M":
            self.interval_pandas = self.interval_pandas[:-1] + "T"
        self.split_time_limits()
        
    def split_time_limits(self):
        self.time_range = pd.date_range(start=self.start_time, 
                                   end=self.end_time,
                                   freq=self.interval_pandas)
        self.time_limits = []
        step = 1000
        for i in range(0, len(self.time_range), step):
            start_time = self.time_range[i]
            if i + step <= len(self.time_range):
                end_time = self.time_range[i+step-1]
            else:
                end_time = self.time_range[-1]
            self.time_limits.append([start_time, end_time])
        
        return len(self.time_limits)
    
    def pull_data(self):
        #print("Making {} API requests".format(len(self.time_limits)))
        data = []
        for i, (start_time, end_time) in tqdm(enumerate(self.time_limits, start=1), total=len(self.time_limits), leave=False):
            start_time = np.int64(start_time.timestamp()*1e3)
            end_time = np.int64(end_time.timestamp()*1e3)
            parameters = [self.symbol, self.interval, start_time, end_time]
            url = self.klines_api.format(*parameters)
            response = requests.get(url)
            if response.status_code == 200:
                rows = json.loads(response.text)
                data.extend(rows)
            if i >= 1000:
                time.sleep(60)
        columns = ["Open-Time", "Open", "High", "Low", "Close", "Volume", "Close-Time", 
                   "Quote-Asset-Value", "Number-of-Trades", 
                   "Taker-Buy-Base-Asset-Volume", "Taker-Buy-Quote-Asset-Volume", "Ignore"]
        if len(data) > 0:
            data = pd.DataFrame(data, columns=columns)
        else:
            return -1
        
        data = self.clean_data(data)
         
        return data
    
    def clean_data(self, data):
        data['Open-Time'] = data['Open-Time'] // 1e3 # ms to s
        data['Close-Time'] = data['Close-Time'] // 1e3 # ms to s
        data['Open-Time'] = pd.to_datetime(data['Open-Time'], unit='s')
        data['Close-Time'] = pd.to_datetime(data['Close-Time'], unit='s')
        
        date_str_format = "%Y-%m-%dT%H:%M:%S"
        open_time_range = list(pd.date_range(start=data['Open-Time'].min(), 
                                             end=data['Open-Time'].max(),
                                             freq=self.interval_pandas))
        open_time_range = [datetime.strftime(t, date_str_format) for t in open_time_range]
        
        open_time_available = [datetime.strftime(pd.Timestamp(t), date_str_format) for t in data['Open-Time'].values]
        open_time_not_available = sorted(list(set(open_time_range) - set(open_time_available)))

        
        columns = ["Open-Time", "Open", "High", "Low", "Close", "Volume", "Close-Time", 
                   "Quote-Asset-Value", "Number-of-Trades", 
                   "Taker-Buy-Base-Asset-Volume", "Taker-Buy-Quote-Asset-Volume", "Ignore"]
        new_data = pd.DataFrame(columns=columns)
        new_data.loc[:, 'Open-Time'] = open_time_not_available
        new_data['Open-Time'] = pd.to_datetime(new_data['Open-Time'], format=date_str_format)
        
        def find_close_time(open_time):
            close_time = pd.date_range(start=open_time, periods=2, freq=self.interval)[-1]
            close_time = close_time - timedelta(seconds=1)
            return close_time
        new_data['Close-Time'] = new_data['Open-Time'].apply(find_close_time)

        data = pd.concat([data, new_data])
        data = data.reset_index(drop=True)
        data = data.sort_values(by=['Open-Time'])
        
        return data

Overwriting PyBinance.py


In [9]:
base_coin = "BNB"
quote_coin = "BTC"
start_time = "2020-01-01T00:00:00"
end_time = "2020-03-01T00:00:00"
interval = "1h"

In [10]:
import importlib
importlib.reload(PyBinance)
puller = PyBinance.BinanceKlinesPuller(base_coin=base_coin, 
                             quote_coin=quote_coin, 
                             start_time=start_time, 
                             end_time=end_time, 
                             interval=interval)
data = puller.pull_data()
print(data.shape)
data.head()

(1441, 12)


,Open-Time,Open,High,Low,Close,Volume,Close-Time,Quote-Asset-Value,Number-of-Trades,Taker-Buy-Base-Asset-Volume,Taker-Buy-Quote-Asset-Volume,Ignore
0,2020-01-01 00:00:00,0.00190670,0.00191000,0.00190520,0.00190780,42127.96000000,2020-01-01 00:59:59,80.38105032,1101,28144.60000000,53.70409308,0
1,2020-01-01 01:00:00,0.00190870,0.00191580,0.00190790,0.00191190,47430.27000000,2020-01-01 01:59:59,90.63450302,1521,28212.86000000,53.90915168,0
2,2020-01-01 02:00:00,0.00191210,0.00191450,0.00190890,0.00191210,35778.05000000,2020-01-01 02:59:59,68.39849054,789,15766.36000000,30.14153545,0
3,2020-01-01 03:00:00,0.00191230,0.00191250,0.00190790,0.00191160,37005.78000000,2020-01-01 03:59:59,70.70972710,775,10962.89000000,20.94506953,0
4,2020-01-01 04:00:00,0.00191170,0.00191250,0.00190880,0.00191140,50233.45000000,2020-01-01 04:59:59,96.02731775,782,21698.03000000,41.48363685,0


In [11]:
data[data.isna().any(axis=1)]

,Open-Time,Open,High,Low,Close,Volume,Close-Time,Quote-Asset-Value,Number-of-Trades,Taker-Buy-Base-Asset-Volume,Taker-Buy-Quote-Asset-Volume,Ignore
1435,2020-02-09 02:00:00,NaN,NaN,NaN,NaN,NaN,2020-02-09 02:59:59,NaN,NaN,NaN,NaN,NaN
1436,2020-02-19 12:00:00,NaN,NaN,NaN,NaN,NaN,2020-02-19 12:59:59,NaN,NaN,NaN,NaN,NaN
1437,2020-02-19 13:00:00,NaN,NaN,NaN,NaN,NaN,2020-02-19 13:59:59,NaN,NaN,NaN,NaN,NaN
1438,2020-02-19 14:00:00,NaN,NaN,NaN,NaN,NaN,2020-02-19 14:59:59,NaN,NaN,NaN,NaN,NaN
1439,2020-02-19 15:00:00,NaN,NaN,NaN,NaN,NaN,2020-02-19 15:59:59,NaN,NaN,NaN,NaN,NaN
1440,2020-02-19 16:00:00,NaN,NaN,NaN,NaN,NaN,2020-02-19 16:59:59,NaN,NaN,NaN,NaN,NaN


## 3. Pull Data

In [12]:
start_time = "2018-01-01T00:00:00"
end_time = "2020-04-30T00:00:00"
interval = "1h"

In [13]:
for row_id, row in tqdm(top_symbols.iterrows(), total=len(top_symbols)):
    base_coin = row['Base-Coin']
    quote_coin = row['Quote-Coin']
    
    puller = PyBinance.BinanceKlinesPuller(base_coin=base_coin, 
                                 quote_coin=quote_coin, 
                                 start_time=start_time, 
                                 end_time=end_time, 
                                 interval=interval)
    data = puller.pull_data()
    if isinstance(data, pd.DataFrame):
        #print(data.shape)
        data_path = "Data/OHLCV/{}_{}_{}.csv".format(base_coin, quote_coin, interval)
        data.to_csv(data_path, index=False)
    #time.sleep(60)